# Build a variational autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

The decoder part of the variational encoder can then be used to generate new reg numbers.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegDvlaVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegDvlaVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 150
actual number of validation samples = 1000
actual number of test samples = 1000


### Variational autoencoder

Design the architecture

In [3]:
latent_dim = 60
intermediate_dim = 500
original_dim = vec_length

### Define the encoder
inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
h = layers.Dense(intermediate_dim, activation='relu')(h)
h = layers.Dense(intermediate_dim, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
# x = layers.Dense(intermediate_dim, activation='relu')(x)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

Do some training

In [4]:
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=256,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

Epoch 1/200
40/40 [==============================] - 3s 74ms/step - loss: 83.0595 - val_loss: 64.5616
Epoch 2/200
40/40 [==============================] - 2s 62ms/step - loss: 56.6591 - val_loss: 50.8798
Epoch 3/200
40/40 [==============================] - 3s 65ms/step - loss: 47.6530 - val_loss: 44.8483
Epoch 4/200
40/40 [==============================] - 3s 64ms/step - loss: 43.0738 - val_loss: 41.5518
Epoch 5/200
40/40 [==============================] - 3s 78ms/step - loss: 40.4101 - val_loss: 39.3294
Epoch 6/200
40/40 [==============================] - 3s 80ms/step - loss: 38.4164 - val_loss: 37.6255
Epoch 7/200
40/40 [==============================] - 3s 79ms/step - loss: 36.9035 - val_loss: 36.1980
Epoch 8/200
40/40 [==============================] - 3s 79ms/step - loss: 35.5538 - val_loss: 34.9296
Epoch 9/200
40/40 [==============================] - 3s 83ms/step - loss: 34.2439 - val_loss: 33.7235
Epoch 10/200
40/40 [==============================] - 3s 63ms/step - loss: 33.0593

40/40 [==============================] - 3s 76ms/step - loss: 8.9211 - val_loss: 9.2314
Epoch 77/200
40/40 [==============================] - 3s 85ms/step - loss: 8.8392 - val_loss: 9.1947
Epoch 78/200
40/40 [==============================] - 4s 89ms/step - loss: 8.7772 - val_loss: 9.2687
Epoch 79/200
40/40 [==============================] - 3s 85ms/step - loss: 8.7348 - val_loss: 9.1303
Epoch 80/200
40/40 [==============================] - 4s 88ms/step - loss: 8.6245 - val_loss: 9.0534
Epoch 81/200
40/40 [==============================] - 3s 87ms/step - loss: 8.5569 - val_loss: 8.9369
Epoch 82/200
40/40 [==============================] - 3s 84ms/step - loss: 8.4797 - val_loss: 8.8315
Epoch 83/200
40/40 [==============================] - 3s 82ms/step - loss: 8.4179 - val_loss: 8.8051
Epoch 84/200
40/40 [==============================] - 3s 77ms/step - loss: 8.3638 - val_loss: 8.7353
Epoch 85/200
40/40 [==============================] - 3s 71ms/step - loss: 8.2705 - val_loss: 8.6086
Epo

Evaluation

In [5]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41JVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
accuracy = 0.956


Use decoder as generator

In [7]:
generated_regs = decoder.predict(np.random.random((5, latent_dim)))
print([v.recover(x) for x in generated_regs])

['OD65XCZ', 'PS35ECZ', 'PY25HEZ', 'HD35EZZ', 'KD65XCZ']


### Avoid generating any training data

There is a chance that a random generation of data might produce one of the actual registration numbers used in training. However, any input registrations are represented as a probability distribution in the latent space; so one could require that any sample vector used in the latent space to be 'sufficiently far' down the tails of all the recorded distributions.



In [20]:
# Get the distributions for each reg number in the data
all_strs = train_strs + val_strs + test_strs
all_vecs = np.array([v.vectorize(x) for x in all_strs])
means, log_sigmas, _ = encoder.predict(all_vecs)
distn_dict = {x: (y,z) for x, y, z in zip(all_strs, means, np.exp(log_sigmas))}  # dict<reg no: (mean, sigma)>
distn_dict['YK66BIQ']
# for a multivariate distribution ~ N(mean, Var) valued in the latent space, 
# mean is the first vector below, and Var is a diagonal matrix, the square root
# of the diagonal entries being given in the second vector
# If the KL loss function has done its job, the distributions should be close to N(0,1)

(array([ 0.15826891, -0.27110144,  0.30188718, -0.15465607,  0.33122134,
        -0.44798145, -0.19661106,  0.3471145 , -0.08936064,  0.1528472 ,
        -0.18759339, -0.3611234 , -0.41070172,  0.03854167,  0.545109  ,
         0.08736099,  0.07961144, -0.36333036, -0.1425544 ,  0.46732378,
        -0.13139188,  0.32754397, -0.15484984, -0.266874  ,  0.38397917,
        -0.36799505,  0.19269603,  0.20504415,  0.04999157, -0.6796633 ,
         0.24879506, -0.3446435 , -0.1613031 ,  0.3423661 , -0.47717273,
         0.00082036, -0.32902792, -0.03171447,  0.44848847, -0.05533407,
        -0.2571784 , -0.0274188 , -0.16290736,  0.3581546 ,  0.44313708,
         0.21607345,  0.10931547, -0.38737673, -0.73970467, -0.35162017,
        -0.22475925,  0.14248486,  0.13281366, -0.2875902 ,  0.20552972,
        -0.1801414 ,  0.02014389, -0.38789982,  0.02563861, -0.3007253 ],
       dtype=float32),
 array([0.9515451 , 0.96728563, 0.91418004, 0.968771  , 0.9112545 ,
        0.8749249 , 0.9602814 , 